In [11]:
import pandas as pd

# Create a list of aspects and companies
aspects = [
    'protect the soil from erosion and degradation', 'efficient use of water resources in the production process', 
    'use of energy-efficient equipment in the production process', 'use of natural detergents for equipment', 
    'use of environmentally friendly wine packaging', 'reducing the use of plastic packaging in production',
    'to reduce winemaking waste', 'to reduce the use of animal additives in wine production',
    'to reduce SO2 emissions', 'reducing the impact of chemical sprays'
]

# Create the initial empty data with companies as columns
data = {
    'Aspect': aspects,
    'Mattia Rivetti': [5, 5, 3, 2, 4, 3, 0, 2, 4, 5],  # Sample data based on the color codes you've mentioned
    'I Vignaioli di S Stefano ssa': [0, 4, 3, 0, 3, 4, 2, 3, 3, 4],
    'AZ. Paolo Berutti': [0, 5, 3, 2, 4, 0, 1, 0, 0, 3]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Save to an Excel file
output_path = "C:/Users/39327/Downloads/Sofi/winery_technology_scores.xlsx"
#if there is no directory make it
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
    
#df.to_excel(output_path, index=False)
# 
df.head()


,Aspect,Mattia Rivetti,I Vignaioli di S Stefano ssa,AZ. Paolo Berutti
0,protect the soil from erosion and degradation,5,0,0
1,efficient use of water resources in the produc...,5,4,5
2,use of energy-efficient equipment in the produ...,3,3,3
3,use of natural detergents for equipment,2,0,2
4,use of environmentally friendly wine packaging,4,3,4


In [13]:
df.to_excel('winery_technology_scores.xlsx', index=False)

In [3]:
!pip install openpyxl

In [1]:
import os
from PIL import Image
import random

# Path to Cat and Dog Dataset
data_dir = r"C:/Users/39327/Downloads/archive (1)/training_set/training_set/"
categories = ['cats', 'dogs']  # cat = 1, dog = 0

# Create list of all data with labels (1 for Cat, 0 for Dog)
data = []
for category in categories:
    label = 1 if category == 'cat' else 0
    category_dir = os.path.join(data_dir, category)
    for filename in os.listdir(category_dir):
        img_path = os.path.join(category_dir, filename)
        data.append({"image": img_path, "label": label})

# Shuffle the data
random.shuffle(data)

# Read single image and label
def read_single(data):
    ent = random.choice(data)  # Choose random entry
    img = Image.open(ent["image"]).convert("RGB")
    img = img.resize((1024, 1024))  # Resize to 1024x1024
    img = np.array(img)
    label = ent["label"]
    return img, label

# Read a batch of images and labels
def read_batch(data, batch_size=4):
    images = []
    labels = []
    for _ in range(batch_size):
        img, label = read_single(data)
        images.append(img)
        labels.append(label)
    return images, np.array(labels)


In [ ]:
# Modify predictor to output binary class (1 for Cat, 0 for Dog)
for itr in range(100000):
    with torch.cuda.amp.autocast():  # Cast to mixed precision
        images, labels = read_batch(data, batch_size=4)  # Load data batch
        if len(labels) == 0: continue  # Ignore empty batches

        predictor.set_image_batch(images)  # Apply SAM image encoder to the image
        
        # Get image embeddings
        image_embeddings = predictor.model.image_encoder(torch.tensor(images).cuda())
        
        # Forward pass through the SAM mask decoder (adapted for classification)
        logits = predictor.model.sam_mask_decoder(image_embeddings=image_embeddings)

        # Classification output (binary logits, one per image)
        preds = torch.sigmoid(logits[:, 0])  # Apply sigmoid to get probabilities for binary classification
        
        # Ground truth labels
        gt_labels = torch.tensor(labels, dtype=torch.float32).cuda()

        # Binary Cross-Entropy Loss for classification
        bce_loss = torch.nn.functional.binary_cross_entropy(preds, gt_labels)

        # Backpropagation
        predictor.model.zero_grad()  # Empty gradients
        scaler.scale(bce_loss).backward()  # Backpropagate
        scaler.step(optimizer)
        scaler.update()  # Update model

        if itr % 1000 == 0:
            torch.save(predictor.model.state_dict(), "model_cat_dog.torch")  # Save model

        # Print results
        mean_accuracy = (preds.round() == gt_labels).float().mean().item()
        print(f"Iteration {itr}: Accuracy = {mean_accuracy}")
